In [1]:
# Test environment
import torch; print('PyTorch:', torch.__version__, 'CUDA:', torch.cuda.is_available())
if torch.cuda.is_available() :
    print(torch.cuda.get_device_name(0))

PyTorch: 2.4.1+cpu CUDA: False


In [2]:
from matcha.models.matcha_tts import MatchaTTS; print('MatchaTTS OK')

MatchaTTS OK


In [3]:
from phonemizer import phonemize; print('Phonemizer OK')
import librosa; print('Librosa OK')

Phonemizer OK
Librosa OK
